In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install mido
import numpy as np
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model

In [2]:
notes = np.load("/content/drive/MyDrive/Major Project/datasets/maestro_notes.npy")
times = np.load("/content/drive/MyDrive/Major Project/datasets/maestro_times.npy")
notes = list(notes)
times = list(times)

In [3]:
print(notes[100:110])
times[100:110]

[56, 72, 56, 68, 51, 68, 63, 51, 72, 44]


[0.03229166666666666,
 0.0020833333333333333,
 0.05520833333333333,
 0.004166666666666667,
 0.007291666666666667,
 0.046875,
 0.009375,
 0.026041666666666668,
 0.005208333333333333,
 0.07604166666666666]

In [5]:
pitchnames = sorted(set(item for item in notes))
print(pitchnames)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
{21: 0, 22: 1, 23: 2, 24: 3, 25: 4, 26: 5, 27: 6, 28: 7, 29: 8, 30: 9, 31: 10, 32: 11, 33: 12, 34: 13, 35: 14, 36: 15, 37: 16, 38: 17, 39: 18, 40: 19, 41: 20, 42: 21, 43: 22, 44: 23, 45: 24, 46: 25, 47: 26, 48: 27, 49: 28, 50: 29, 51: 30, 52: 31, 53: 32, 54: 33, 55: 34, 56: 35, 57: 36, 58: 37, 59: 38, 60: 39, 61: 40, 62: 41, 63: 42, 64: 43, 65: 44, 66: 45, 67: 46, 68: 47, 69: 48, 70: 49, 71: 50, 72: 51, 73: 52, 74: 53, 75: 54, 76: 55, 77: 56, 78: 57, 79: 58, 80: 59, 81: 60, 82: 61, 83: 62, 84: 63, 85: 64, 86: 65, 87: 66, 88: 67, 89: 68, 90: 69, 91: 70, 92: 71, 93: 72, 94: 73, 95: 74, 96: 75, 97: 76, 98: 77, 99: 78, 100: 79, 101: 8

In [4]:
note = []
octaves = []
for a in notes:
  n = (a - 11) % 12
  octave = int((a - 11) / 12)
  note.append(n)
  octaves.append(octave)
notes = note
del note

In [8]:
notes[100:110]

[10, 2, 10, 10, 5, 10, 5, 5, 2, 10]

In [9]:
octaves[100:110]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [5]:
from sklearn.preprocessing import MinMaxScaler

def scaling(data, feature_range = (0, 1)):
  scaler = MinMaxScaler(feature_range)
  scaler.fit(np.array(data).reshape(-1, 1))
  transformed_data = list(scaler.transform(np.array(data).reshape(-1, 1)))
  return transformed_data, scaler

In [6]:
scaled_notes, notes_scaler = scaling(notes)
scaled_octaves, octaves_scaler = scaling(octaves)
log_times = [max(0, -np.log10(time + 0.0001)) for time in times]
scaled_times, times_scaler = scaling(log_times)

In [7]:
del notes
del octaves
del times

In [8]:
print(scaled_notes[100:110])
print(scaled_octaves[100:110])
scaled_times[100:110]

[array([0.81818182]), array([0.09090909]), array([0.81818182]), array([0.81818182]), array([0.36363636]), array([0.81818182]), array([0.36363636]), array([0.36363636]), array([0.09090909]), array([0.81818182])]
[array([0.375]), array([0.625]), array([0.375]), array([0.5]), array([0.375]), array([0.5]), array([0.5]), array([0.375]), array([0.625]), array([0.25])]


[array([0.37239168]),
 array([0.66521999]),
 array([0.31430236]),
 array([0.59247782]),
 array([0.53281441]),
 array([0.3320333]),
 array([0.5058552]),
 array([0.39566668]),
 array([0.56876045]),
 array([0.27959441])]

In [22]:
tym = times_scaler.inverse_transform(scaled_times[100:110])

In [23]:
print([10**(-time) for time in tym])

[array([0.03239167]), array([0.00218333]), array([0.05530833]), array([0.00426667]), array([0.00739167]), array([0.046975]), array([0.009475]), array([0.02614167]), array([0.00530833]), array([0.07614167])]


In [ ]:
notes_with_times = []
i = 0
for note in list(scaled_notes):
    temp = []
    temp.append(note)
    temp.append(list(scaled_octaves[i]))
    temp.append(list(scaled_times[i]))
    notes_with_times.append(temp)
    i += 1
notes_with_times = np.array(notes_with_times).reshape(-1, 3)
notes_with_times.shape

In [9]:
#np.save("/content/drive/MyDrive/Major Project/arch/maestro_notes_octaves_times.npy", notes_with_times)
notes_with_times = np.load("/content/drive/MyDrive/Major Project/arch/maestro_notes_octaves_times.npy")
notes_with_times.shape

(14080328, 3)

In [10]:
del scaled_notes
del scaled_octaves
del scaled_times

In [11]:
notes_with_times = [list(note) for note in notes_with_times]

X = []

# number of notes in a sequence
n_prev = 100
#for i in range(0, len(notes_with_times)-n_prev, 10):
  #X.append(notes_with_times[i : i + n_prev])
  
for i in range(0, len(notes_with_times) - n_prev, n_prev):
  X.append(notes_with_times[i : i + n_prev])

In [12]:
X = np.array(X)
print(X[1, 0:10])
print(X.shape)
#np.random.shuffle(X)
#print(X[0,22,:])
#print(X.shape)

[[0.81818182 0.375      0.37239168]
 [0.09090909 0.625      0.66521999]
 [0.81818182 0.375      0.31430236]
 [0.81818182 0.5        0.59247782]
 [0.36363636 0.375      0.53281441]
 [0.81818182 0.5        0.3320333 ]
 [0.36363636 0.5        0.5058552 ]
 [0.36363636 0.375      0.39566668]
 [0.09090909 0.625      0.56876045]
 [0.81818182 0.25       0.27959441]]
(140803, 100, 3)


In [13]:
del notes_with_times

In [ ]:
def generator():
    gen_input = Input(shape = (None, 100))
    X = LSTM(256, return_sequences = True)(gen_input)
    X = BatchNormalization(momentum = 0.8)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = LSTM(128, return_sequences = True)(X)
    X = BatchNormalization(momentum = 0.8)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = Dense(64)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = BatchNormalization(momentum = 0.8)(X)
    X = Dropout(0.5)(X)
    out = Dense(3, activation = "tanh")(X)
    model = Model(inputs = gen_input, outputs = out)
    return model

In [ ]:
def discriminator():
    disc_input = Input(shape = (100, 3))
    X = LSTM(128, return_sequences = True)(disc_input)
    X = LeakyReLU(alpha = 0.2)(X)
    X = LSTM(64, return_sequences = True)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    X = Flatten()(X)
    X = Dense(128)(X)
    X = LeakyReLU(alpha = 0.2)(X)
    out = Dense(1, activation = "sigmoid")(X)
    model = Model(inputs = disc_input, outputs = out)
    return model

In [ ]:
def disc_with_gen(generator, discriminator):
    inputs = Input(shape = (None, 100))
    generated = generator(inputs)
    outputs = discriminator(generated)
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
optimizer = Adam(0.0002, 0.5)
#optimizer = SGD(lr = 0.03)
d = discriminator()
d.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
g = generator()

In [ ]:
d.trainable = False
combined = disc_with_gen(g, d)
combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [32]:
combined.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, None, 100)]       0         
_________________________________________________________________
model_4 (Functional)         (None, None, 3)           572931    
_________________________________________________________________
model_3 (Functional)         (None, 1)                 936449    
Total params: 1,509,380
Trainable params: 572,035
Non-trainable params: 937,345
_________________________________________________________________


In [ ]:
from matplotlib import pyplot as plt
assert not np.any(np.isnan(X))

In [ ]:
disc_loss = []
disc_loss_epoch = []
gen_loss = []
gen_loss_epoch = []

In [ ]:
batch_size = 128

In [ ]:
idx = list(range(0, batch_size, 1))
real_data = X[idx]
real_data.shape

(128, 100, 3)

In [ ]:
def train(epochs, num_of_batches, batch_size = batch_size):
    for epoch in range(epochs):
        a = 0
        b = batch_size
        for num in range(num_of_batches):
          idx = list(range(a, b, 1))
          real_data = X[idx]

          noise = np.random.normal(0, 1, (batch_size, 100, 100))
        
          generated = g.predict(noise)
          d_loss_real = d.train_on_batch(real_data, np.ones((batch_size, 1)))
          #print("d_loss_real: ", d_loss_real)
          d_loss_fake = d.train_on_batch(generated, np.zeros((batch_size, 1)))
          #print("d_loss_fake: ",d_loss_fake)
          d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
          print("%d d_loss: " %epoch,d_loss)

          for _ in range(2):            
            noise = np.random.normal(0, 1, (batch_size, 100, 100))
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
          print("%d g_loss: " %epoch, g_loss)

          print("\n")

          disc_loss.append(d_loss[0])
          gen_loss.append(g_loss[0])

          a = a + batch_size
          b = b + batch_size
            
        if epoch % 1 == 0:
            print ("Epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], 100*g_loss[1]))
            print("\n")
            disc_loss_epoch.append(d_loss[0])
            gen_loss_epoch.append(g_loss[0])
            save_model()
            save_weights()
            save_losses()
            checkpoint = tf.train.Checkpoint(g)
            checkpoint.save('/content/drive/MyDrive/Major Project/arch/checkpoints/training_checkpoints')
            
    plt.plot(disc_loss, c='red')
    plt.plot(gen_loss, c='blue')
    plt.title("GAN Loss per batch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.savefig('/content/drive/MyDrive/Major Project/output/GAN_Loss_per_Batch_final.png', transparent=False)
    plt.close()

    plt.plot(disc_loss_epoch, c='red')
    plt.plot(gen_loss_epoch, c='blue')
    plt.title("GAN Loss per Epoch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('/content/drive/MyDrive/Major Project/output/GAN_Loss_per_Epoch_final.png', transparent=False)
    plt.close()

In [ ]:
def save_model():
  d.trainable = False
  combined.save("/content/drive/MyDrive/Major Project/arch/models/combined_model_time_maestro.h5")
  d.trainable = True
  g.save("/content/drive/MyDrive/Major Project/arch/models/gen_model_time_maestro.h5")
  d.save("/content/drive/MyDrive/Major Project/arch/models/disc_model_time_maestro.h5")

In [ ]:
def save_weights():
  g.save_weights("/content/drive/MyDrive/Major Project/arch/weights/gen_model_weights_time_maestro.h5")
  d.save_weights("/content/drive/MyDrive/Major Project/arch/weights/disc_model_weights_time_maestro.h5")
  combined.save_weights("/content/drive/MyDrive/Major Project/arch/weights/combined_model_weights_time_maestro.h5")

In [ ]:
def save_losses():
  np.save("/content/drive/MyDrive/Major Project/arch/losses/disc_loss.npy", np.array(disc_loss))
  np.save("/content/drive/MyDrive/Major Project/arch/losses/disc_loss_epoch.npy", np.array(disc_loss_epoch))
  np.save("/content/drive/MyDrive/Major Project/arch/losses/gen_loss.npy", np.array(gen_loss))
  np.save("/content/drive/MyDrive/Major Project/arch/losses/gen_loss_epoch.npy", np.array(gen_loss_epoch))

In [ ]:
print(real_data.shape)
generated = g.predict(np.random.normal(0, 1, (batch_size, 100, 100)))
print(generated.shape)
#idx = list(range(0, batch_size, 1))
#print(real_data[idx])

(128, 100, 3)
(128, 100, 3)


In [ ]:
train(epochs = 20, num_of_batches = int(X.shape[0]/batch_size))

Streaming output truncated to the last 5000 lines.


18 d_loss:  [1.35572882e-04 1.00000000e+00]
18 g_loss:  [5.009596861782484e-06, 1.0]


18 d_loss:  [1.92621754e-04 1.00000000e+00]
18 g_loss:  [1.2265464647498447e-05, 1.0]


18 d_loss:  [1.67642954e-05 1.00000000e+00]
18 g_loss:  [1.349418016616255e-05, 1.0]


18 d_loss:  [1.25705272e-04 1.00000000e+00]
18 g_loss:  [1.2620258530660067e-05, 1.0]


18 d_loss:  [4.46651538e-05 1.00000000e+00]
18 g_loss:  [2.7352261895430274e-06, 1.0]


18 d_loss:  [9.45739452e-04 1.00000000e+00]
18 g_loss:  [5.9958356359857135e-06, 1.0]


18 d_loss:  [6.26705503e-05 1.00000000e+00]
18 g_loss:  [7.954530701681506e-06, 1.0]


18 d_loss:  [7.29029614e-04 1.00000000e+00]
18 g_loss:  [1.0427069355500862e-05, 1.0]


18 d_loss:  [0.00215639 1.        ]
18 g_loss:  [4.925353096041363e-06, 1.0]


18 d_loss:  [2.05658425e-05 1.00000000e+00]
18 g_loss:  [5.293932190397754e-06, 1.0]


18 d_loss:  [3.73776111e-04 1.00000000e+00]
18 g_loss:  [5.8408777476870455e-06,

In [ ]:
#g.load_weights("/content/drive/MyDrive/Major Project/saved_weights/Model_weights_time_128_100_1_500k_data_repeating_50k.h5")

In [14]:
g = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/arch/models/gen_model_time_maestro.h5")
d = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/arch/models/disc_model_time_maestro.h5")
combined = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/arch/models/combined_model_time_maestro.h5")

In [15]:
disc_loss = list(np.load("/content/drive/MyDrive/Major Project/arch/losses/disc_loss.npy"))
disc_loss_epoch = list(np.load("/content/drive/MyDrive/Major Project/arch/losses/disc_loss.npy"))
gen_loss = list(np.load("/content/drive/MyDrive/Major Project/arch/losses/disc_loss.npy"))
gen_loss_epoch = list(np.load("/content/drive/MyDrive/Major Project/arch/losses/disc_loss.npy"))

In [16]:
pred = g.predict(np.random.normal(0, 1, (1, 100, 100)))
d.predict(pred)

array([[3.1456232e-10]], dtype=float32)

In [17]:
d.predict(X[100, :, :].reshape(1, 100, 3))

array([[0.99999666]], dtype=float32)

In [18]:
prediction = g.predict(np.random.normal(0, 1, (1, 100, 100)))
print(prediction)

[[[0.97437865 0.33197945 0.4369525 ]
  [0.6028104  0.7355444  0.52062523]
  [0.5370897  0.76255715 0.57022977]
  [0.36136267 0.78145355 0.60088825]
  [0.33051956 0.67593354 0.48940745]
  [0.3205381  0.5442506  0.40498972]
  [0.6057557  0.34322038 0.360693  ]
  [0.6903055  0.30086547 0.45260066]
  [0.5849665  0.39964545 0.5125755 ]
  [0.55199844 0.44098836 0.5359974 ]
  [0.71414673 0.4028451  0.53059494]
  [0.55471057 0.47228134 0.54526573]
  [0.72393686 0.41715968 0.5326672 ]
  [0.67427456 0.43612003 0.55295056]
  [0.69803464 0.4393332  0.5373086 ]
  [0.6937967  0.46702737 0.5194702 ]
  [0.49647886 0.5220219  0.5496237 ]
  [0.6013336  0.487286   0.55527216]
  [0.56991947 0.5005621  0.55611867]
  [0.62951005 0.497549   0.55140513]
  [0.6488826  0.4838048  0.56870395]
  [0.6380772  0.5140643  0.5468042 ]
  [0.6231653  0.5150372  0.5701349 ]
  [0.66805285 0.48859367 0.57816654]
  [0.64925456 0.51982105 0.5558736 ]
  [0.6795134  0.51117295 0.56366205]
  [0.6095596  0.5395053  0.561346  ]
 

In [19]:
predicted_notes = []
predicted_octaves = []
predicted_times = []
for p in prediction.reshape(-1, 3):
    predicted_notes.append(p[0])
    predicted_octaves.append(p[1])
    predicted_times.append(p[2])
print(np.array(predicted_notes).shape)
print(np.array(predicted_octaves).shape)
print(np.array(predicted_times).shape)
print(predicted_notes)
print(predicted_octaves)
print(predicted_times)

(100,)
(100,)
(100,)
[0.97437865, 0.6028104, 0.5370897, 0.36136267, 0.33051956, 0.3205381, 0.6057557, 0.6903055, 0.5849665, 0.55199844, 0.71414673, 0.55471057, 0.72393686, 0.67427456, 0.69803464, 0.6937967, 0.49647886, 0.6013336, 0.56991947, 0.62951005, 0.6488826, 0.6380772, 0.6231653, 0.66805285, 0.64925456, 0.6795134, 0.6095596, 0.66441756, 0.6576042, 0.6742303, 0.7319052, 0.57686013, 0.6042528, 0.66446435, 0.66311103, 0.63875806, 0.5980574, 0.5901489, 0.45850542, 0.63058126, 0.63232803, 0.67509615, 0.6422802, 0.7193055, 0.72552115, 0.490214, 0.7202377, 0.74984616, 0.5960984, 0.6523214, 0.7292737, 0.6129733, 0.60778236, 0.6236454, 0.7312016, 0.6020082, 0.6615889, 0.612399, 0.7395379, 0.6616125, 0.6641846, 0.65645075, 0.7100581, 0.67939824, 0.61658585, 0.7796351, 0.72875065, 0.5311282, 0.69446504, 0.7166363, 0.75492656, 0.5152859, 0.777052, 0.7173015, 0.7070658, 0.66806513, 0.73619306, 0.5417867, 0.63422847, 0.72535825, 0.708963, 0.6567665, 0.64015454, 0.7441751, 0.6391779, 0.5248298,

In [20]:
_notes = notes_scaler.inverse_transform(np.array(predicted_notes).reshape(-1,1))
_notes = [int(i) for i in _notes]
print(_notes)
_octaves = octaves_scaler.inverse_transform(np.array(predicted_octaves).reshape(-1,1))
_octaves = [int(i) for i in _octaves]
print(_octaves)
final_notes = [(12 * _octaves[idx] + note + 11) for idx, note in enumerate(_notes)]
final_notes

[10, 6, 5, 3, 3, 3, 6, 7, 6, 6, 7, 6, 7, 7, 7, 7, 5, 6, 6, 6, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 8, 6, 6, 7, 7, 7, 6, 6, 5, 6, 6, 7, 7, 7, 7, 5, 7, 8, 6, 7, 8, 6, 6, 6, 8, 6, 7, 6, 8, 7, 7, 7, 7, 7, 6, 8, 8, 5, 7, 7, 8, 5, 8, 7, 7, 7, 8, 5, 6, 7, 7, 7, 7, 8, 7, 5, 7, 7, 7, 8, 8, 7, 8, 7, 8, 8, 7, 7, 7, 8]
[2, 5, 6, 6, 5, 4, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 5, 4, 4, 4]


[45,
 77,
 88,
 86,
 74,
 62,
 41,
 42,
 53,
 53,
 54,
 53,
 54,
 54,
 54,
 54,
 64,
 53,
 65,
 53,
 54,
 66,
 65,
 54,
 66,
 66,
 65,
 66,
 66,
 66,
 67,
 65,
 65,
 66,
 66,
 66,
 65,
 65,
 64,
 65,
 65,
 66,
 66,
 66,
 66,
 64,
 66,
 67,
 65,
 66,
 67,
 65,
 65,
 65,
 67,
 65,
 66,
 65,
 67,
 66,
 66,
 66,
 66,
 66,
 65,
 67,
 67,
 76,
 66,
 66,
 67,
 76,
 67,
 66,
 66,
 66,
 67,
 76,
 77,
 66,
 66,
 66,
 66,
 67,
 78,
 76,
 66,
 66,
 66,
 67,
 67,
 78,
 67,
 78,
 67,
 67,
 78,
 66,
 66,
 67]

In [21]:
_times = times_scaler.inverse_transform(np.array(predicted_times).reshape(-1, 1))
_times = list(_times.reshape(100,))
final_times = [10**(-time) for time in _times]
final_times

[0.017872692931026463,
 0.008269877288901285,
 0.005236980116384213,
 0.003948635002335475,
 0.011024786810866681,
 0.023990600702474836,
 0.03607679670756957,
 0.015473813858925656,
 0.008906313361019642,
 0.007178115415882834,
 0.007544322067593249,
 0.006590783764658647,
 0.007401693912536123,
 0.00614041545080612,
 0.007091950654048208,
 0.008358322818453936,
 0.006331477805189717,
 0.006010510389036436,
 0.005963831046103323,
 0.006228442690280623,
 0.005311096619689783,
 0.0064980528805578565,
 0.005241559105039008,
 0.004867812283311884,
 0.005977309016631747,
 0.005563547385958921,
 0.00568350206481757,
 0.004371423214179229,
 0.005104494471802,
 0.006652361997811814,
 0.00626952350918259,
 0.004999756574700622,
 0.004977018679345927,
 0.004603908747884725,
 0.005553044511177946,
 0.006404449982211467,
 0.005830138643305061,
 0.005383814264636063,
 0.004948007522932511,
 0.004432963784948826,
 0.004463905033854751,
 0.004861920698074355,
 0.004630973568514499,
 0.00439960354896

In [22]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
for i in range(100):
    # 147 means note_on
    # 67 is velocity
    if final_notes[i] > 127:
      final_notes[i] = 127
    if final_notes[i] < 0:
      final_notes[i] = 0
    note = np.asarray([147, final_notes[i], 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    time = final_times[i]
    msg.time = round(time * mid.ticks_per_beat * 2)
    if msg.note > 21 or msg.note < 109:
      track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/Major Project/output/LSTM_music_time_check66.mid')
for msg in mid:
  print(msg)

note_on channel=3 note=45 velocity=67 time=0.017708333333333333
note_on channel=3 note=77 velocity=67 time=0.008333333333333333
note_on channel=3 note=88 velocity=67 time=0.005208333333333333
note_on channel=3 note=86 velocity=67 time=0.004166666666666667
note_on channel=3 note=74 velocity=67 time=0.011458333333333333
note_on channel=3 note=62 velocity=67 time=0.02395833333333333
note_on channel=3 note=41 velocity=67 time=0.036458333333333336
note_on channel=3 note=42 velocity=67 time=0.015625
note_on channel=3 note=53 velocity=67 time=0.009375
note_on channel=3 note=53 velocity=67 time=0.007291666666666667
note_on channel=3 note=54 velocity=67 time=0.007291666666666667
note_on channel=3 note=53 velocity=67 time=0.00625
note_on channel=3 note=54 velocity=67 time=0.007291666666666667
note_on channel=3 note=54 velocity=67 time=0.00625
note_on channel=3 note=54 velocity=67 time=0.007291666666666667
note_on channel=3 note=54 velocity=67 time=0.008333333333333333
note_on channel=3 note=64 v

In [23]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
t = 230
track.append(Message('program_change', program = 0, channel = 0, time = 0))
for note in final_notes:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    if t >= 160:
        t -= 2
    msg.time = int(t)
    msg.channel = 0
    track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/Major Project/output/LSTM_music_check66.mid')
for msg in mid:
  print(msg)

program_change channel=0 program=0 time=0
note_on channel=0 note=45 velocity=67 time=0.2375
note_on channel=0 note=77 velocity=67 time=0.23541666666666666
note_on channel=0 note=88 velocity=67 time=0.23333333333333334
note_on channel=0 note=86 velocity=67 time=0.23124999999999998
note_on channel=0 note=74 velocity=67 time=0.22916666666666666
note_on channel=0 note=62 velocity=67 time=0.22708333333333333
note_on channel=0 note=41 velocity=67 time=0.225
note_on channel=0 note=42 velocity=67 time=0.22291666666666665
note_on channel=0 note=53 velocity=67 time=0.22083333333333333
note_on channel=0 note=53 velocity=67 time=0.21875
note_on channel=0 note=54 velocity=67 time=0.21666666666666667
note_on channel=0 note=53 velocity=67 time=0.21458333333333332
note_on channel=0 note=54 velocity=67 time=0.2125
note_on channel=0 note=54 velocity=67 time=0.21041666666666667
note_on channel=0 note=54 velocity=67 time=0.20833333333333334
note_on channel=0 note=54 velocity=67 time=0.20625
note_on channe

In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

mid = MidiFile('/content/drive/MyDrive/Major Project/output/LSTM_music.mid')
#mid = MidiFile('pnowks/corner.mid') 
notes = []

for msg in mid:
    print(msg)

note_on channel=3 note=69 velocity=67 time=0.2375
note_on channel=3 note=46 velocity=67 time=0.23541666666666666
note_on channel=3 note=89 velocity=67 time=0.23333333333333334
note_on channel=3 note=57 velocity=67 time=0.23124999999999998
note_on channel=3 note=58 velocity=67 time=0.22916666666666666
note_on channel=3 note=64 velocity=67 time=0.22708333333333333
note_on channel=3 note=67 velocity=67 time=0.225
note_on channel=3 note=59 velocity=67 time=0.22291666666666665
note_on channel=3 note=53 velocity=67 time=0.22083333333333333
note_on channel=3 note=53 velocity=67 time=0.21875
note_on channel=3 note=58 velocity=67 time=0.21666666666666667
note_on channel=3 note=56 velocity=67 time=0.21458333333333332
note_on channel=3 note=74 velocity=67 time=0.2125
note_on channel=3 note=69 velocity=67 time=0.21041666666666667
note_on channel=3 note=52 velocity=67 time=0.20833333333333334
note_on channel=3 note=59 velocity=67 time=0.20625
note_on channel=3 note=48 velocity=67 time=0.20416666666

In [ ]:
b = [1,2,3,4,5,6,7,8,9]
a = list(np.array(b).reshape(-1,1))
note = [list(v) for v in a]
print(a)

[array([1]), array([2]), array([3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9])]


In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

#mid = MidiFile('/content/drive/MyDrive/Major Project/output/LSTM_music.mid')
mid = MidiFile('corner.mid')
midy = MidiFile()
track = MidiTrack()

start = 0

midy.ticks_per_beat = mid.ticks_per_beat

for msg in mid:
    msg.time = round(msg.time * midy.ticks_per_beat * 2)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        msg.note = prediction[start]
        start += 1
        track.append(msg)
        if start >= len(prediction):
            break
    else:
        track.append(msg)
midy.tracks.append(track)
midy.save("/content/drive/MyDrive/Major Project/output/lstmaa.mid")

In [ ]:
for msg in mid:
    print(msg)

<meta message track_name name='untitled' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=500000 time=0>
<meta message midi_port port=0 time=0>
<meta message track_name name='Piano' time=0>
control_change channel=0 control=0 value=0 time=0
control_change channel=0 control=32 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=64 value=127 time=2.0708333333333333
control_change channel=0 control=64 value=0 time=0.10833333333333334
control_change channel=0 control=64 value=127 time=0.2833333333333333
control_change channel=0 control=67 value=127 time=0.0375
note_on channel=0 note=36 velocity=54 time=0.13333333333333333
note_on channel=0 note=48 velocity=73 time=0.25416666666666665
note_on channel=0 note=55 velocity=52 time=0.5125
note_on channel=0 note=60 velocity=60 time=0.12083333333333333
note_on channel

note_on channel=0 note=54 velocity=0 time=0.008333333333333333
note_on channel=0 note=63 velocity=0 time=0.020833333333333332
note_on channel=0 note=46 velocity=0 time=0.09166666666666666
note_on channel=0 note=63 velocity=73 time=0.03333333333333333
note_on channel=0 note=54 velocity=62 time=0.0125
note_on channel=0 note=58 velocity=62 time=0.008333333333333333
note_on channel=0 note=54 velocity=0 time=0.025
note_on channel=0 note=58 velocity=0 time=0.016666666666666666
note_on channel=0 note=63 velocity=0 time=0.020833333333333332
control_change channel=0 control=64 value=0 time=0.17916666666666667
note_on channel=0 note=44 velocity=64 time=0.0125
note_on channel=0 note=61 velocity=82 time=0.004166666666666667
note_on channel=0 note=56 velocity=70 time=0.004166666666666667
note_on channel=0 note=53 velocity=69 time=0.016666666666666666
control_change channel=0 control=64 value=127 time=0
note_on channel=0 note=53 velocity=0 time=0.075
note_on channel=0 note=56 velocity=0 time=0.00416

note_on channel=0 note=80 velocity=103 time=0.0375
note_on channel=0 note=66 velocity=76 time=0.0125
note_on channel=0 note=64 velocity=72 time=0.008333333333333333
note_on channel=0 note=64 velocity=0 time=0.16666666666666666
note_on channel=0 note=66 velocity=0 time=0.0125
note_on channel=0 note=73 velocity=0 time=0.09583333333333333
note_on channel=0 note=80 velocity=0 time=0.0125
note_on channel=0 note=70 velocity=0 time=0
note_on channel=0 note=61 velocity=83 time=0.07083333333333333
note_on channel=0 note=61 velocity=0 time=0.03333333333333333
note_on channel=0 note=54 velocity=54 time=0.32916666666666666
note_on channel=0 note=54 velocity=0 time=0.08333333333333333
note_on channel=0 note=64 velocity=53 time=0.25416666666666665
note_on channel=0 note=67 velocity=57 time=0.016666666666666666
note_on channel=0 note=70 velocity=62 time=0.1875
note_on channel=0 note=64 velocity=0 time=0.125
note_on channel=0 note=58 velocity=60 time=0
note_on channel=0 note=60 velocity=49 time=0.0125

note_on channel=0 note=63 velocity=0 time=0.10416666666666667
note_on channel=0 note=58 velocity=82 time=0.029166666666666667
note_on channel=0 note=61 velocity=83 time=0
control_change channel=0 control=64 value=0 time=0
note_on channel=0 note=70 velocity=77 time=0.016666666666666666
note_on channel=0 note=73 velocity=82 time=0.004166666666666667
control_change channel=0 control=64 value=127 time=0.075
note_on channel=0 note=70 velocity=0 time=0.041666666666666664
note_on channel=0 note=73 velocity=0 time=0.05
note_on channel=0 note=65 velocity=76 time=0.0125
note_on channel=0 note=58 velocity=0 time=0.0125
note_on channel=0 note=61 velocity=0 time=0.0125
note_on channel=0 note=65 velocity=0 time=0.0375
note_on channel=0 note=63 velocity=83 time=0.06666666666666667
note_on channel=0 note=63 velocity=0 time=0.058333333333333334
note_on channel=0 note=63 velocity=83 time=0.04583333333333333
note_on channel=0 note=63 velocity=0 time=0.06666666666666667
note_on channel=0 note=63 velocity=

note_on channel=0 note=57 velocity=0 time=0.0125
note_on channel=0 note=42 velocity=0 time=0.008333333333333333
control_change channel=0 control=67 value=104 time=0.11666666666666667
control_change channel=0 control=67 value=88 time=0.008333333333333333
control_change channel=0 control=67 value=72 time=0.008333333333333333
control_change channel=0 control=67 value=56 time=0.0125
control_change channel=0 control=67 value=40 time=0
control_change channel=0 control=67 value=24 time=0.020833333333333332
control_change channel=0 control=67 value=0 time=0.004166666666666667
note_on channel=0 note=49 velocity=59 time=0.0875
note_on channel=0 note=57 velocity=54 time=0.3125
note_on channel=0 note=58 velocity=67 time=0.0625
note_on channel=0 note=61 velocity=66 time=0.004166666666666667
note_on channel=0 note=42 velocity=63 time=0.020833333333333332
note_on channel=0 note=61 velocity=0 time=0.0125
note_on channel=0 note=58 velocity=0 time=0.016666666666666666
note_on channel=0 note=57 velocity=

In [ ]:
for msg in midy:
    print(msg)

<meta message track_name name='untitled' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=500000 time=0>
<meta message midi_port port=0 time=0>
<meta message track_name name='Piano' time=0>
control_change channel=0 control=0 value=0 time=0
control_change channel=0 control=32 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=64 value=127 time=2.0708333333333333
control_change channel=0 control=64 value=0 time=0.10833333333333334
control_change channel=0 control=64 value=127 time=0.2833333333333333
control_change channel=0 control=67 value=127 time=0.0375
note_on channel=0 note=69 velocity=54 time=0.13333333333333333
note_on channel=0 note=46 velocity=73 time=0.25416666666666665
note_on channel=0 note=89 velocity=52 time=0.5125
note_on channel=0 note=57 velocity=60 time=0.12083333333333333
note_on channel

In [ ]:
midy.ticks_per_beat

120

In [ ]:
int(X.shape[0]/batch_size)

407